In [5]:
from langchain.tools import StructuredTool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
import pandas as pd
from openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = #ADD API Key

In [6]:
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool
from langchain_core.pydantic_v1 import BaseModel, Field
import pandas as pd

# Load and save functions
def load_inventory(filepath='inventory.csv'):
    return pd.read_csv(filepath)

def save_inventory(df, filepath='inventory.csv'):
    df.to_csv(filepath, index=False)

# Adjust quantity tool
class AdjustQuantityInput(BaseModel):
    item: str = Field(description="The name of the item to adjust in the inventory. Convert to singular form and correct grammar")
    quantity: int = Field(description="The quantity to adjust (positive to add, negative to remove)")

def adjust_quantity(item: str, quantity: int) -> str:
    df = load_inventory()
    if item in df['item'].values:
        current_quantity = df.loc[df['item'] == item, 'quantity'].values[0]
        new_quantity = current_quantity + quantity
        if new_quantity < 0:
            df.loc[df['item'] == item, 'quantity'] = 0
            result = f"Warning: {item} cannot have a negative quantity. Set to 0 instead."
        else:
            df.loc[df['item'] == item, 'quantity'] = new_quantity
            result = f"Updated {item} by {quantity}. New quantity is {new_quantity}."
    else:
        df = pd.concat([df,pd.DataFrame([{'item': item, 'quantity': max(quantity, 0)}])], ignore_index=True)
        result = f"Added {item} to inventory with quantity {max(quantity, 0)}."
    save_inventory(df)
    return result

adjust_tool = StructuredTool.from_function(
    func=adjust_quantity,
    name="AdjustQuantity",
    description="Adjust the quantity of an item in the inventory (add or remove).",
    args_schema=AdjustQuantityInput,
)

# Set quantity tool
class SetQuantityInput(BaseModel):
    item: str = Field(description="The name of the item to set in the inventory")
    quantity: int = Field(description="The total quantity to set for the item")

def set_quantity(item: str, quantity: int) -> str:
    df = load_inventory()
    if item in df['item'].values:
        df.loc[df['item'] == item, 'quantity'] = quantity
        result = f"Set the quantity of {item} to {quantity}."
    else:
        df = pd.concat([df,pd.DataFrame([{'item': item, 'quantity': max(quantity, 0)}])], ignore_index=True)
        result = f"Added {item} to inventory with quantity {quantity}."
    save_inventory(df)
    return result

set_tool = StructuredTool.from_function(
    func=set_quantity,
    name="SetQuantity",
    description="Set the total quantity of an item in the inventory.",
    args_schema=SetQuantityInput,
)

In [7]:

prompt = ChatPromptTemplate.from_template(
    "You are a fridge inventory manager. Your job is to update the inventory based on user inputs. "
    "Use the tools provided to make the necessary changes. Think about which tool is the most appropriate."
    "\n\nCurrent thoughts:\n{agent_scratchpad}\n\n"
    "User input: {input}\n"
    "Action: "
)

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a fridge inventory manager. Your job is to update the inventory based on user inputs. "
            "Use the tools provided to make the necessary changes. Think about which tool is the most appropriate.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [8]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

llm = ChatOpenAI(model="gpt-4o", temperature=0)
tools=[adjust_tool, set_tool]
llm_with_tools = llm.bind_tools(tools)

# Create the agent
#agent = create_openai_functions_agent(llm, tools=[adjust_tool, set_tool], prompt=prompt)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)


# Create the executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Example input
response = agent_executor.invoke({"input": "I am putting 2 yoghurts, 5 apples and 2 eggs in the fridge"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `AdjustQuantity` with `{'item': 'yoghurt', 'quantity': 2}`


Added yoghurt to inventory with quantity 2.
Invoking: `AdjustQuantity` with `{'item': 'apple', 'quantity': 5}`


Updated apple by 5. New quantity is 8.
Invoking: `AdjustQuantity` with `{'item': 'egg', 'quantity': 2}`


Added egg to inventory with quantity 2.I have updated the inventory:
- Added 2 yoghurts.
- Added 5 apples.
- Added 2 eggs.

Is there anything else you would like to update?

> Finished chain.
{'input': 'I am putting 2 yoghurts, 5 apples and 2 eggs in the fridge', 'output': 'I have updated the inventory:\n- Added 2 yoghurts.\n- Added 5 apples.\n- Added 2 eggs.\n\nIs there anything else you would like to update?'}


In [10]:
response = agent_executor.invoke({"input": "I put 3 apples in the fridge and also took out 1 egg"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `AdjustQuantity` with `{'item': 'apple', 'quantity': 3}`


Updated apple by 3. New quantity is 8.
Invoking: `AdjustQuantity` with `{'item': 'egg', 'quantity': -1}`


Updated egg by -1. New quantity is 1.I've updated the inventory:

- Added 3 apples. The new quantity is 8.
- Removed 1 egg. The new quantity is 1.

> Finished chain.
{'input': 'I put 3 apples in the fridge and also took out 1 egg', 'output': "I've updated the inventory:\n\n- Added 3 apples. The new quantity is 8.\n- Removed 1 egg. The new quantity is 1."}
